# 第3回講義 宿題

## 課題
今Lessonで学んだことを元に, MNISTのファッション版 (Fashion MNIST, クラス数10) を多層パーセプトロン (MLP) によって分類してみましょう.

Fashion MNISTの詳細については以下のリンクを参考にしてください.

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 85%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **MLPのアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap03`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap03 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します.
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します.
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します. これを最終的な評価とします.

### データの読み込み
- この部分は修正しないでください。

In [1]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('/root/userspace/public/chap03/data/x_train.npy')
    y_train = np.load('/root/userspace/public/chap03/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap03/data/x_test.npy')

    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

### 多層パーセプトロンの実装

In [3]:
#%%writefile /root/userspace/chap03/materials/submission_code.py

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x_train, y_train, x_test = load_fashionmnist()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1000)

def np_log(x):
    # WRITE ME
    return np.log(np.clip(x, 1e-10, x))

def relu(x):
    # WRITE ME
    return np.maximum(x, 0)

def deriv_relu(x):
    # WRITE ME
    return (x > 0).astype(x.dtype)

def softmax(x):
    # WRITE ME
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def deriv_softmax(x):
    # WRITE ME
    return softmax() * (1 - softmax(x))

class Dense:
    # WRITE ME
    def __init__(self, in_dim, out_dim, function, deriv_function):
        self.W = np.random.uniform(low=-0.08, high=0.08, size=(in_dim, out_dim)).astype('float64')
        self.b = np.zeros(out_dim).astype('float64')
        self.function = function
        self.deriv_function = deriv_function
        
        self.x = None
        self.u = None
        self.dW = None
        self.db = None
        
        self.params_idxs = np.cumsum([self.W.size, self.b.size])
        
    def __call__(self, x):
        self.x = x
        self.u = np.matmul(self.x, self.W) + self.b
        return self.function(self.u)
    
    def b_prop(self, delta, W):
        self.delta = self.deriv_function(self.u) * np.matmul(delta, W.T) # for the delta of layers other than output layer
        return self.delta
    
    def compute_grad(self):
        batch_size = self.delta.shape[0]
        self.dW = np.matmul(self.x.T, self.delta) / batch_size
        self.db = np.matmul(np.ones(batch_size), self.delta) / batch_size
        
    def get_params(self):
        return np.concatenate([self.W.ravel(), self.b], axis=0)
    
    def set_params(self, params):
        _W, _b = np.split(params, self.params_idxs)[:-1]
        self.W = _W.reshape(self.W.shape)
        self.b = _b
        
    def get_grads(self):
        return np.concatenate([self.dW.ravel(), self.db], axis=0)

def f_props(layers, x):
    # WRITE ME
    for layer in layers:
        x = layer(x) # here calls Dense.__call__(x)
    return x

def b_props(layers, delta):
    # WRITE ME
    batch_size = delta.shape[0]
    for i,layer in enumerate(layers[::-1]):
        if i == 0: # for the case of the output layer
            layer.delta = delta # delta = y - t
            layer.compute_grad()
        else: # for the case of layers other than the output layer
            delta = layer.b_prop(delta, W) # here calls Dens.b_prop(delta, W) which return self.delta = self.deriv_function(self.u) * np.matmul(delta, W.T)
            layer.compute_grad()
        W = layer.W
        

def update_params(layers, eps):
    # WRITE ME
    for layer in layers:
        layer.W -= eps * layer.dW
        layer.b -= eps * layer.db

layers = [
    # WRITE ME
    Dense(784, 100, relu, deriv_relu),
    Dense(100, 100, relu, deriv_relu),
    Dense(100, 10, softmax, deriv_softmax)
]

def train(x, t, eps):
    # WRITE ME
    # forward propagation
    y = f_props(layers, x)
    
    # calculate error
    cost = (-t * np_log(y)).sum(axis=1).mean()
    
    # back propagation
    delta = y - t # for the output layer
    b_props(layers, delta) # the output layer uses delata above directly, other layers use self.delta = self.deriv_function(self.u) * np.matmul(delta, W.T) via Dens.b_prop(delta, W)
    
    # update parameters
    update_params(layers, eps)
    
    return cost
    
    
def valid(x, t):
    # WRITE ME
    # forward propagation
    y = f_props(layers, x)
    
    cost = (-t * np_log(y)).sum(axis=1).mean() # this is the same as the calculation of error in train_mnist()
    
    return cost, y

def test(x):
    # forward propagation
    y = f_props(layers, x)
    
    return y
    
    
    

for epoch in range(50):
    x_train, y_train = shuffle(x_train, y_train)
    # オンライン学習
    # WRITE ME
    for x,t in zip(x_train, y_train):
        cost = train(x[None, :], t[None, :], eps=0.01)
        
    cost, y_pred = valid(x_valid, y_valid)
    accuracy = accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    
    print("EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}".format(epoch+1, cost, accuracy))

    
y_pred = test(x_test)
y_pred = np.argmax(y_pred, axis=1)

submission = pd.Series(y_pred, name='label')
submission.to_csv('/root/userspace/chap03/materials/submission_pred.csv', header=True, index_label='id')

EPOCH: 1, Valid Cost: 0.414, Valid Accuracy: 0.845
EPOCH: 2, Valid Cost: 0.410, Valid Accuracy: 0.839
EPOCH: 3, Valid Cost: 0.356, Valid Accuracy: 0.865
EPOCH: 4, Valid Cost: 0.390, Valid Accuracy: 0.857
EPOCH: 5, Valid Cost: 0.405, Valid Accuracy: 0.857
EPOCH: 6, Valid Cost: 0.328, Valid Accuracy: 0.879
EPOCH: 7, Valid Cost: 0.373, Valid Accuracy: 0.851
EPOCH: 8, Valid Cost: 0.329, Valid Accuracy: 0.882
EPOCH: 9, Valid Cost: 0.323, Valid Accuracy: 0.875
EPOCH: 10, Valid Cost: 0.322, Valid Accuracy: 0.882
EPOCH: 11, Valid Cost: 0.316, Valid Accuracy: 0.895
EPOCH: 12, Valid Cost: 0.307, Valid Accuracy: 0.889
EPOCH: 13, Valid Cost: 0.338, Valid Accuracy: 0.886
EPOCH: 14, Valid Cost: 0.336, Valid Accuracy: 0.881
EPOCH: 15, Valid Cost: 0.350, Valid Accuracy: 0.879
EPOCH: 16, Valid Cost: 0.367, Valid Accuracy: 0.876
EPOCH: 17, Valid Cost: 0.399, Valid Accuracy: 0.865
EPOCH: 18, Valid Cost: 0.337, Valid Accuracy: 0.888
EPOCH: 19, Valid Cost: 0.364, Valid Accuracy: 0.884
EPOCH: 20, Valid Cost